# Notebook 2
# Data join from different tables and it's EDA

### Objectives of this notebook
* Join significant features from different  data tables
* Explore data
* Prepare data for next step feature selection step
* Save data for next feature selection step

In [1]:
import os
import pandas as pd
import numpy as np
import shap
from collections import Counter
import warnings
import scipy
import joblib
from sklearn import metrics
from scipy.stats import mannwhitneyu, chi2_contingency
from sklearn.metrics import make_scorer, silhouette_score
from sklearn.model_selection import StratifiedKFold,KFold,RandomizedSearchCV
import lightgbm
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict
import sweetviz as sv
from IPython.display import display, IFrame,Image
from sklearn.metrics import mean_squared_error, r2_score
from matplotlib import pyplot as plt
import seaborn as sns
import analysis_functions
from IPython.display import display, IFrame
from sklearn.metrics import (
    roc_auc_score,
    roc_curve,
    recall_score,
    confusion_matrix,
    ConfusionMatrixDisplay,
    classification_report,
    balanced_accuracy_score,
    f1_score,
)
from sklearn.utils.class_weight import compute_class_weight
import matplotlib.pyplot as plt

warnings.filterwarnings("ignore")


%matplotlib inline

In [25]:
df_bureau = pd.read_csv("prepared_data/preprocessed_bureau_data_final.csv")
df_bureau

,SK_ID_CURR,CREDIT_ACTIVE_Active_count_Bureau,CREDIT_ACTIVE_Closed_count_Bureau,CREDIT_ACTIVE_Sold_count_Bureau,CREDIT_CURRENCY_currency 1_count_Bureau,CREDIT_TYPE_Consumer credit_count_Bureau,CREDIT_TYPE_Credit card_count_Bureau,CREDIT_TYPE_Other_count_Bureau,DAYS_CREDIT_Bureau,CREDIT_DAY_OVERDUE_Bureau,CNT_CREDIT_PROLONG_Bureau,AMT_CREDIT_SUM_OVERDUE_Bureau,DAYS_CREDIT_UPDATE_Bureau
0,100001,3.0,4.0,0.0,7.0,7.0,0.0,0.0,-735.000000,0.0,0.000000,0.0,-93.142857
1,100002,2.0,6.0,0.0,8.0,4.0,4.0,0.0,-874.000000,0.0,0.000000,0.0,-499.875000
2,100003,1.0,3.0,0.0,4.0,2.0,2.0,0.0,-1400.750000,0.0,0.000000,0.0,-816.000000
3,100004,0.0,2.0,0.0,2.0,2.0,0.0,0.0,-867.000000,0.0,0.000000,0.0,-532.000000
4,100005,2.0,1.0,0.0,3.0,2.0,1.0,0.0,-190.666667,0.0,0.000000,0.0,-54.333333
...,...,...,...,...,...,...,...,...,...,...,...,...,...
305806,456249,2.0,11.0,0.0,13.0,9.0,3.0,1.0,-1667.076923,0.0,0.000000,0.0,-1064.538462
305807,456250,2.0,1.0,0.0,3.0,2.0,1.0,0.0,-862.000000,0.0,0.000000,0.0,-60.333333
305808,456253,2.0,2.0,0.0,4.0,3.0,1.0,0.0,-867.500000,0.0,0.000000,0.0,-253.250000
305809,456254,0.0,1.0,0.0,1.0,1.0,0.0,0.0,-1104.000000,0.0,0.000000,0.0,-401.000000


In [26]:
df_pcash = pd.read_csv("prepared_data/final-pos_cash.csv")
df_pcash

,SK_ID_CURR,NAME_CONTRACT_STATUS_Active_count_pos_cash,NAME_CONTRACT_STATUS_Completed_count_pos_cash,NAME_CONTRACT_STATUS_Other_count_pos_cash,MONTHS_BALANCE_pos_cash,SK_DPD_pos_cash,SK_DPD_DEF_pos_cash
0,100001,7.0,2.0,0.0,-72.555556,0.777778,0.777778
1,100002,19.0,0.0,0.0,-10.000000,0.000000,0.000000
2,100003,26.0,2.0,0.0,-43.785714,0.000000,0.000000
3,100004,3.0,1.0,0.0,-25.500000,0.000000,0.000000
4,100005,9.0,1.0,0.0,-20.000000,0.000000,0.000000
...,...,...,...,...,...,...,...
337247,456251,7.0,1.0,0.0,-5.000000,0.000000,0.000000
337248,456252,6.0,1.0,0.0,-79.000000,0.000000,0.000000
337249,456253,15.0,2.0,0.0,-79.235294,0.294118,0.294118
337250,456254,20.0,0.0,0.0,-5.550000,0.000000,0.000000


In [27]:
df_installments = pd.read_csv("prepared_data/HCredit_installments_final.csv")
df_installments

,SK_ID_CURR,AMT_INSTALMENT_mean_HCredit_installments,DAYS_INSTALMENT_mean_HCredit_installments,NUM_INSTALMENT_NUMBER_mean_HCredit_installments,NUM_INSTALMENT_VERSION_mean_HCredit_installments
0,100001,5885.132143,-2187.714286,2.714286,1.142857
1,100002,11559.247105,-295.000000,10.000000,1.052632
2,100003,64754.586000,-1378.160000,5.080000,1.040000
3,100004,7096.155000,-754.000000,2.000000,1.333333
4,100005,6240.205000,-586.000000,5.000000,1.111111
...,...,...,...,...,...
339582,456251,7492.924286,-120.000000,4.000000,1.142857
339583,456252,10069.867500,-2391.000000,3.500000,1.000000
339584,456253,4399.707857,-2372.928571,4.785714,1.000000
339585,456254,10239.832895,-142.263158,5.263158,1.000000


In [28]:
df_previuosapp = pd.read_csv("prepared_data/HCredit_Previuos_App_final.csv")
df_previuosapp

,SK_ID_CURR,NAME_CONTRACT_TYPE_Consumer loans_count_HCredit_PApp,NAME_CONTRACT_TYPE_Revolving loans_count_HCredit_PApp,NAME_CONTRACT_TYPE_XNA_count_HCredit_PApp,NAME_CASH_LOAN_PURPOSE_XAP_count_HCredit_PApp,NAME_CASH_LOAN_PURPOSE_XNA_count_HCredit_PApp,NAME_CONTRACT_STATUS_Canceled_count_HCredit_PApp,NAME_CONTRACT_STATUS_Refused_count_HCredit_PApp,NAME_PAYMENT_TYPE_XNA_count_HCredit_PApp,CODE_REJECT_REASON_Other_count_HCredit_PApp,...,NAME_YIELD_GROUP_middle_count_HCredit_PApp,PRODUCT_COMBINATION_Other_count_HCredit_PApp,PRODUCT_COMBINATION_POS household with interest_count_HCredit_PApp,PRODUCT_COMBINATION_POS mobile with interest_count_HCredit_PApp,AMT_APPLICATION_HCredit_PApp,AMT_CREDIT_HCredit_PApp,FLAG_LAST_APPL_PER_CONTRACT_HCredit_PApp,NFLAG_LAST_APPL_IN_DAY_HCredit_PApp,DAYS_DECISION_HCredit_PApp,SELLERPLACE_AREA_HCredit_PApp
0,100001,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,24835.500,23787.00,1.0,1.0,-1740.000,23.000
1,100002,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,179055.000,179055.00,1.0,1.0,-606.000,500.000
2,100003,2.0,0.0,0.0,2.0,1.0,0.0,0.0,1.0,0.0,...,2.0,2.0,1.0,0.0,435436.500,484191.00,1.0,1.0,-1305.000,533.000
3,100004,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,0.0,0.0,24282.000,20106.00,1.0,1.0,-815.000,30.000
4,100005,1.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,1.0,22308.750,20076.75,1.0,1.0,-536.000,18.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
338852,456251,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,40455.000,40455.00,1.0,1.0,-273.000,30.000
338853,456252,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,57595.500,56821.50,1.0,1.0,-2497.000,190.000
338854,456253,2.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,2.0,24162.750,20625.75,1.0,1.0,-2380.000,22.000
338855,456254,2.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,1.0,121317.750,134439.75,1.0,1.0,-299.500,1578.000


In [29]:
df_main = pd.read_csv("prepared_data/significant_train_final.csv")
df_main

,SK_ID_CURR,AMT_INCOME_TOTAL,AMT_CREDIT,REGION_POPULATION_RELATIVE,DAYS_REGISTRATION,TARGET,CNT_CHILDREN,DAYS_BIRTH,DAYS_ID_PUBLISH,FLAG_EMP_PHONE,...,FLAG_OWN_CAR,FLAG_OWN_REALTY,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,OCCUPATION_TYPE,WEEKDAY_APPR_PROCESS_START,ORGANIZATION_TYPE
0,100002,202500.0,406597.5,0.018801,-3648.0,1,0,-9461,-2120,1,...,N,Y,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,Laborers,WEDNESDAY,Business Entity Type 3
1,100003,270000.0,1293502.5,0.003541,-1186.0,0,0,-16765,-291,1,...,N,N,Family,State servant,Higher education,Married,House / apartment,Core staff,MONDAY,School
2,100004,67500.0,135000.0,0.010032,-4260.0,0,0,-19046,-2531,1,...,Y,Y,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,Laborers,MONDAY,Government
3,100006,135000.0,312682.5,0.008019,-9833.0,0,0,-19005,-2437,1,...,N,Y,Unaccompanied,Working,Secondary / secondary special,Civil marriage,House / apartment,Laborers,WEDNESDAY,Business Entity Type 3
4,100007,121500.0,513000.0,0.028663,-4311.0,0,0,-19932,-3458,1,...,N,Y,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,Core staff,THURSDAY,Religion
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307506,456251,157500.0,254700.0,0.032561,-8456.0,0,0,-9327,-1982,1,...,N,N,Unaccompanied,Working,Secondary / secondary special,Separated,With parents,Sales staff,THURSDAY,Services
307507,456252,72000.0,269550.0,0.025164,-4388.0,0,0,-20775,-4090,0,...,N,Y,Unaccompanied,Pensioner,Secondary / secondary special,Widow,House / apartment,NaN,MONDAY,XNA
307508,456253,153000.0,677664.0,0.005002,-6737.0,0,0,-14966,-5150,1,...,N,Y,Unaccompanied,Working,Higher education,Separated,House / apartment,Managers,THURSDAY,School
307509,456254,171000.0,370107.0,0.005313,-2562.0,1,0,-11961,-931,1,...,N,Y,Unaccompanied,Commercial associate,Secondary / secondary special,Married,House / apartment,Laborers,WEDNESDAY,Business Entity Type 1


In [30]:
df_CCard = pd.read_csv("prepared_data/Credit-card_payments_final.csv")
df_CCard

,SK_ID_CURR,NAME_CONTRACT_STATUS_Active_count_HCredit_CCard,NAME_CONTRACT_STATUS_Completed_count_HCredit_CCard,MONTHS_BALANCE_HCredit_CCard,AMT_BALANCE_HCredit_CCard,AMT_DRAWINGS_CURRENT_HCredit_CCard,AMT_INST_MIN_REGULARITY_HCredit_CCard,AMT_PAYMENT_TOTAL_CURRENT_HCredit_CCard,AMT_RECEIVABLE_PRINCIPAL_HCredit_CCard,AMT_RECIVABLE_HCredit_CCard,AMT_TOTAL_RECEIVABLE_HCredit_CCard,CNT_DRAWINGS_CURRENT_HCredit_CCard,CNT_INSTALMENT_MATURE_CUM_HCredit_CCard,SK_DPD_DEF_HCredit_CCard
0,100006,6.0,0.0,-3.5,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,100011,74.0,0.0,-38.5,54482.111149,2432.432432,3956.221849,4520.067568,52402.088919,54433.179122,54433.179122,0.054054,25.767123,0.000000
2,100013,96.0,0.0,-48.5,18159.919219,5953.125000,1454.539551,6817.172344,17255.559844,18101.079844,18101.079844,0.239583,18.719101,0.010417
3,100021,7.0,10.0,-10.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,100023,8.0,0.0,-7.5,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103553,456244,36.0,5.0,-21.0,131834.730732,26842.388049,6514.200000,32720.544878,127608.373537,130767.060732,130767.060732,1.365854,13.600000,0.000000
103554,456246,8.0,0.0,-5.5,13136.731875,15199.256250,1439.150625,15554.340000,12883.016250,12897.894375,12897.894375,2.500000,3.500000,0.000000
103555,456247,95.0,0.0,-49.0,23216.396211,2149.506474,1414.704789,4115.878105,22100.653895,23128.243105,23128.243105,0.147368,26.494737,0.021053
103556,456248,23.0,0.0,-13.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [31]:
df_final = pd.merge(df_main, df_bureau, on='SK_ID_CURR', how='left')

In [32]:
df_final = pd.merge(df_final, df_CCard, on='SK_ID_CURR', how='left')

In [33]:
df_final = pd.merge(df_final, df_installments, on='SK_ID_CURR', how='left')

In [34]:
df_final = pd.merge(df_final, df_pcash, on='SK_ID_CURR', how='left')

In [35]:
df_final = pd.merge(df_final, df_previuosapp, on='SK_ID_CURR', how='left')

In [36]:
df_final.columns.tolist()

['SK_ID_CURR',
 'AMT_INCOME_TOTAL',
 'AMT_CREDIT',
 'REGION_POPULATION_RELATIVE',
 'DAYS_REGISTRATION',
 'TARGET',
 'CNT_CHILDREN',
 'DAYS_BIRTH',
 'DAYS_ID_PUBLISH',
 'FLAG_EMP_PHONE',
 'FLAG_WORK_PHONE',
 'FLAG_PHONE',
 'REGION_RATING_CLIENT',
 'REGION_RATING_CLIENT_W_CITY',
 'HOUR_APPR_PROCESS_START',
 'REG_REGION_NOT_LIVE_REGION',
 'REG_REGION_NOT_WORK_REGION',
 'REG_CITY_NOT_LIVE_CITY',
 'REG_CITY_NOT_WORK_CITY',
 'LIVE_CITY_NOT_WORK_CITY',
 'FLAG_DOCUMENT_2',
 'FLAG_DOCUMENT_3',
 'FLAG_DOCUMENT_6',
 'FLAG_DOCUMENT_8',
 'FLAG_DOCUMENT_9',
 'FLAG_DOCUMENT_11',
 'FLAG_DOCUMENT_13',
 'FLAG_DOCUMENT_14',
 'FLAG_DOCUMENT_15',
 'FLAG_DOCUMENT_16',
 'FLAG_DOCUMENT_18',
 'NAME_CONTRACT_TYPE',
 'CODE_GENDER',
 'FLAG_OWN_CAR',
 'FLAG_OWN_REALTY',
 'NAME_TYPE_SUITE',
 'NAME_INCOME_TYPE',
 'NAME_EDUCATION_TYPE',
 'NAME_FAMILY_STATUS',
 'NAME_HOUSING_TYPE',
 'OCCUPATION_TYPE',
 'WEEKDAY_APPR_PROCESS_START',
 'ORGANIZATION_TYPE',
 'CREDIT_ACTIVE_Active_count_Bureau',
 'CREDIT_ACTIVE_Closed_coun

In [37]:
df_final

,SK_ID_CURR,AMT_INCOME_TOTAL,AMT_CREDIT,REGION_POPULATION_RELATIVE,DAYS_REGISTRATION,TARGET,CNT_CHILDREN,DAYS_BIRTH,DAYS_ID_PUBLISH,FLAG_EMP_PHONE,...,NAME_YIELD_GROUP_middle_count_HCredit_PApp,PRODUCT_COMBINATION_Other_count_HCredit_PApp,PRODUCT_COMBINATION_POS household with interest_count_HCredit_PApp,PRODUCT_COMBINATION_POS mobile with interest_count_HCredit_PApp,AMT_APPLICATION_HCredit_PApp,AMT_CREDIT_HCredit_PApp,FLAG_LAST_APPL_PER_CONTRACT_HCredit_PApp,NFLAG_LAST_APPL_IN_DAY_HCredit_PApp,DAYS_DECISION_HCredit_PApp,SELLERPLACE_AREA_HCredit_PApp
0,100002,202500.0,406597.5,0.018801,-3648.0,1,0,-9461,-2120,1,...,0.0,1.0,0.0,0.0,179055.000,179055.00,1.0,1.0,-606.000000,500.000000
1,100003,270000.0,1293502.5,0.003541,-1186.0,0,0,-16765,-291,1,...,2.0,2.0,1.0,0.0,435436.500,484191.00,1.0,1.0,-1305.000000,533.000000
2,100004,67500.0,135000.0,0.010032,-4260.0,0,0,-19046,-2531,1,...,1.0,1.0,0.0,0.0,24282.000,20106.00,1.0,1.0,-815.000000,30.000000
3,100006,135000.0,312682.5,0.008019,-9833.0,0,0,-19005,-2437,1,...,1.0,6.0,1.0,0.0,272203.260,291695.50,1.0,1.0,-272.444444,894.222222
4,100007,121500.0,513000.0,0.028663,-4311.0,0,0,-19932,-3458,1,...,3.0,4.0,1.0,1.0,150530.250,166638.75,1.0,1.0,-1222.833333,409.166667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307506,456251,157500.0,254700.0,0.032561,-8456.0,0,0,-9327,-1982,1,...,0.0,0.0,0.0,1.0,40455.000,40455.00,1.0,1.0,-273.000000,30.000000
307507,456252,72000.0,269550.0,0.025164,-4388.0,0,0,-20775,-4090,0,...,0.0,1.0,0.0,0.0,57595.500,56821.50,1.0,1.0,-2497.000000,190.000000
307508,456253,153000.0,677664.0,0.005002,-6737.0,0,0,-14966,-5150,1,...,0.0,0.0,0.0,2.0,24162.750,20625.75,1.0,1.0,-2380.000000,22.000000
307509,456254,171000.0,370107.0,0.005313,-2562.0,1,0,-11961,-931,1,...,0.0,0.0,1.0,1.0,121317.750,134439.75,1.0,1.0,-299.500000,1578.000000


In [38]:
df_target_0 = df_final[df_final['TARGET'] == 0]
df_target_1 = df_final[df_final['TARGET'] == 1]

In [39]:
train_comparison = sv.compare(df_target_0, df_target_1)

                                             |          | [  0%]   00:00 -> (? left)

In [40]:
train_comparison.show_html("Train_final_target_comparison.html")

Report Train_final_target_comparison.html was generated! NOTEBOOK/COLAB USERS: the web browser MAY not pop up, regardless, the report IS saved in your notebook/colab files.


In [2]:
html_file_path = "Train_final_target_comparison.html"

IFrame(html_file_path, width="100%", height=1000)

In [21]:
threshold = 0.53 * len(df_final)

df_final_preprocessed = df_final.dropna(axis=1, thresh=threshold)

df_final_preprocessed

,SK_ID_CURR,AMT_INCOME_TOTAL,AMT_CREDIT,REGION_POPULATION_RELATIVE,DAYS_REGISTRATION,TARGET,CNT_CHILDREN,DAYS_BIRTH,DAYS_ID_PUBLISH,FLAG_EMP_PHONE,...,NAME_YIELD_GROUP_middle_count_HCredit_PApp,PRODUCT_COMBINATION_Other_count_HCredit_PApp,PRODUCT_COMBINATION_POS household with interest_count_HCredit_PApp,PRODUCT_COMBINATION_POS mobile with interest_count_HCredit_PApp,AMT_APPLICATION_HCredit_PApp,AMT_CREDIT_HCredit_PApp,FLAG_LAST_APPL_PER_CONTRACT_HCredit_PApp,NFLAG_LAST_APPL_IN_DAY_HCredit_PApp,DAYS_DECISION_HCredit_PApp,SELLERPLACE_AREA_HCredit_PApp
0,100002,202500.0,406597.5,0.018801,-3648.0,1,0,-9461,-2120,1,...,0.0,1.0,0.0,0.0,179055.000,179055.00,1.0,1.0,-606.000000,500.000000
1,100003,270000.0,1293502.5,0.003541,-1186.0,0,0,-16765,-291,1,...,2.0,2.0,1.0,0.0,435436.500,484191.00,1.0,1.0,-1305.000000,533.000000
2,100004,67500.0,135000.0,0.010032,-4260.0,0,0,-19046,-2531,1,...,1.0,1.0,0.0,0.0,24282.000,20106.00,1.0,1.0,-815.000000,30.000000
3,100006,135000.0,312682.5,0.008019,-9833.0,0,0,-19005,-2437,1,...,1.0,6.0,1.0,0.0,272203.260,291695.50,1.0,1.0,-272.444444,894.222222
4,100007,121500.0,513000.0,0.028663,-4311.0,0,0,-19932,-3458,1,...,3.0,4.0,1.0,1.0,150530.250,166638.75,1.0,1.0,-1222.833333,409.166667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307506,456251,157500.0,254700.0,0.032561,-8456.0,0,0,-9327,-1982,1,...,0.0,0.0,0.0,1.0,40455.000,40455.00,1.0,1.0,-273.000000,30.000000
307507,456252,72000.0,269550.0,0.025164,-4388.0,0,0,-20775,-4090,0,...,0.0,1.0,0.0,0.0,57595.500,56821.50,1.0,1.0,-2497.000000,190.000000
307508,456253,153000.0,677664.0,0.005002,-6737.0,0,0,-14966,-5150,1,...,0.0,0.0,0.0,2.0,24162.750,20625.75,1.0,1.0,-2380.000000,22.000000
307509,456254,171000.0,370107.0,0.005313,-2562.0,1,0,-11961,-931,1,...,0.0,0.0,1.0,1.0,121317.750,134439.75,1.0,1.0,-299.500000,1578.000000


Dropped features which has more than 53% of missing values

# Fix of categorical variables

In [22]:
df_final_preprocessed = analysis_functions.keep_top_categories(df_final_preprocessed, 'ORGANIZATION_TYPE', 3)

In [24]:
df_final_preprocessed = analysis_functions.keep_top_categories(df_final_preprocessed, 'OCCUPATION_TYPE', 5)

In [25]:
df_final_preprocessed = analysis_functions.keep_top_categories(df_final_preprocessed, 'NAME_FAMILY_STATUS', 3)

In [26]:
df_final_preprocessed = analysis_functions.keep_top_categories(df_final_preprocessed, 'NAME_TYPE_SUITE', 2)

In [27]:
df_final_preprocessed = analysis_functions.keep_top_categories(df_final_preprocessed, 'NAME_INCOME_TYPE', 3)

In [28]:
df_final_preprocessed = analysis_functions.keep_top_categories(df_final_preprocessed, 'NAME_EDUCATION_TYPE', 2)

In [29]:
df_final_preprocessed = analysis_functions.keep_top_categories(df_final_preprocessed, 'NAME_HOUSING_TYPE', 3)

Leaving only those values which has 10% or more records in a dataset

In [30]:
mapping = {'Y': 1, 'N': 0}


df_final_preprocessed['FLAG_OWN_CAR'] = df_final_preprocessed['FLAG_OWN_CAR'].map(mapping)
df_final_preprocessed['FLAG_OWN_REALTY'] = df_final_preprocessed['FLAG_OWN_REALTY'].map(mapping)

In features FLAG_OWN_CAR and FLAG_OWN_REALTY , mapping values to {'Y'= 1, 'N'= 0}

In [33]:
df_final_preprocessed

,SK_ID_CURR,AMT_INCOME_TOTAL,AMT_CREDIT,REGION_POPULATION_RELATIVE,DAYS_REGISTRATION,TARGET,CNT_CHILDREN,DAYS_BIRTH,DAYS_ID_PUBLISH,FLAG_EMP_PHONE,...,NAME_YIELD_GROUP_middle_count_HCredit_PApp,PRODUCT_COMBINATION_Other_count_HCredit_PApp,PRODUCT_COMBINATION_POS household with interest_count_HCredit_PApp,PRODUCT_COMBINATION_POS mobile with interest_count_HCredit_PApp,AMT_APPLICATION_HCredit_PApp,AMT_CREDIT_HCredit_PApp,FLAG_LAST_APPL_PER_CONTRACT_HCredit_PApp,NFLAG_LAST_APPL_IN_DAY_HCredit_PApp,DAYS_DECISION_HCredit_PApp,SELLERPLACE_AREA_HCredit_PApp
0,100002,202500.0,406597.5,0.018801,-3648.0,1,0,-9461,-2120,1,...,0.0,1.0,0.0,0.0,179055.000,179055.00,1.0,1.0,-606.000000,500.000000
1,100003,270000.0,1293502.5,0.003541,-1186.0,0,0,-16765,-291,1,...,2.0,2.0,1.0,0.0,435436.500,484191.00,1.0,1.0,-1305.000000,533.000000
2,100004,67500.0,135000.0,0.010032,-4260.0,0,0,-19046,-2531,1,...,1.0,1.0,0.0,0.0,24282.000,20106.00,1.0,1.0,-815.000000,30.000000
3,100006,135000.0,312682.5,0.008019,-9833.0,0,0,-19005,-2437,1,...,1.0,6.0,1.0,0.0,272203.260,291695.50,1.0,1.0,-272.444444,894.222222
4,100007,121500.0,513000.0,0.028663,-4311.0,0,0,-19932,-3458,1,...,3.0,4.0,1.0,1.0,150530.250,166638.75,1.0,1.0,-1222.833333,409.166667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307506,456251,157500.0,254700.0,0.032561,-8456.0,0,0,-9327,-1982,1,...,0.0,0.0,0.0,1.0,40455.000,40455.00,1.0,1.0,-273.000000,30.000000
307507,456252,72000.0,269550.0,0.025164,-4388.0,0,0,-20775,-4090,0,...,0.0,1.0,0.0,0.0,57595.500,56821.50,1.0,1.0,-2497.000000,190.000000
307508,456253,153000.0,677664.0,0.005002,-6737.0,0,0,-14966,-5150,1,...,0.0,0.0,0.0,2.0,24162.750,20625.75,1.0,1.0,-2380.000000,22.000000
307509,456254,171000.0,370107.0,0.005313,-2562.0,1,0,-11961,-931,1,...,0.0,0.0,1.0,1.0,121317.750,134439.75,1.0,1.0,-299.500000,1578.000000


In [34]:
df_final_preprocessed.to_csv("combined_train_df.csv", index=False)